# RC_calc

#将不同的损伤状态赋予相应的维修等级

In [25]:
import sys
import numpy as np
import pandas as pd
import os
from more_itertools import locate

import json
import pickle


In [2]:
# 获取当前目录路径
current_dir = os.getcwd()

# 定义子目录名
subdir_name = 'Output_treads_RR'

# 创建子目录
subdir_path = os.path.join(current_dir, subdir_name)
if not os.path.exists(subdir_path):
    os.makedirs(subdir_path)

In [3]:
DMG_input = 'DMG_sample_R.csv' #pelicun output 经过前处理，去掉了倒塌和不可修复的情况
RCtable_input = 'Repair_Class_Table_R.csv'
DL_summary_input = 'DL_summary.csv'
output_path = subdir_path

In [4]:
#determine if DMG file contains "nan" entries or not
DMG = pd.read_csv(DMG_input, low_memory=False, header=None)

In [5]:
if DMG[4:].isnull().values.any()==True:
    sys.exit("DMG.csv file contains blank entries")

In [6]:
DMG

,0,1,2,3,4,5,6,7,8,9,...,3522,3523,3524,3525,3526,3527,3528,3529,3530,3531
0,cmp,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,...,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.50.11.013l,D.50.12.023c,D.50.12.023l
1,loc,1,1,10,10,11,11,12,12,13,...,6,60,60,60,7,8,9,28,28,28
2,dir,1,2,1,2,1,2,1,2,1,...,0,0,0,0,0,0,0,0,0,0
3,ds,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,0,0,0,0
4,0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,205.99999999999997,36.0,0.0,0.0,205.99999999999997,205.99999999999997,205.99999999999997,4.0,39.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999,1995,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,205.99999999999997,36.0,0.0,0.0,205.99999999999997,205.99999999999997,205.99999999999997,4.0,39.0,8.0
2000,1996,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,205.99999999999997,36.0,0.0,0.0,205.99999999999997,205.99999999999997,205.99999999999997,4.0,39.0,8.0
2001,1997,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,205.99999999999997,36.0,0.0,0.0,205.99999999999997,205.99999999999997,205.99999999999997,4.0,39.0,8.0
2002,1998,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,205.99999999999997,36.0,0.0,0.0,205.99999999999997,205.99999999999997,205.99999999999997,4.0,39.0,8.0


In [7]:
RC_table = pd.read_csv(RCtable_input)
FG_all = RC_table["Fragility"]
RC_all = RC_table[["RC-DS0","RC-DS1","RC-DS2","RC-DS3", "RC-DS4", "RC-DS5", "RC-DS6", "RC-DS7", "RC-DS8", "RC-DS9", "RC-DS10", "RC-DS11", "RC-DS12", "RC-DS13", "RC-DS14", "RC-DS15"]]
RS_all = RC_table["Repair Sequence"]

PG = DMG.loc[1, 1:] #performance groups indicating the location of fragility 由易损性编号和楼层决定

In [9]:
FG = DMG.loc[0, 1:] #fragility groups 由易损性编号决定
n_PG = len(FG)
n_PG

3531

In [10]:
#check if all fragilities are defined in the repair class table
RS_FG =[]
for i in range(n_PG):
    indx_FG = FG_all[FG_all==FG[i+1]].index
    if indx_FG.values.size==0:
        sys.exit("Fragility not found. Check Repair_Class_Table.csv file")
    RS_FG.append(RS_all[indx_FG[0]])

RS = list(set(RS_FG))
RS.sort()

story_FG = []
for i in range(n_PG):
    story_FG.append((PG[i+1]))    
story = list(set(story_FG))
story.sort()

In [ ]:
story

In [11]:
#determine indices for collpase, irreparable, and repairable realizations    
DL_summary=pd.read_csv(DL_summary_input)
cases_collapse = DL_summary["collapses/collapsed"]
indx_collapse = cases_collapse[cases_collapse!=0].index
cases_irreparable = DL_summary["reconstruction/irreparable"]
indx_irreparable = cases_irreparable[cases_irreparable!=0].index
indx_all = DL_summary["#Num"]
indx_repairable =  indx_all.drop(indx_collapse.union(indx_irreparable))

In [20]:
indx_collapse

Index([], dtype='int64')

In [21]:
indx_irreparable

Index([], dtype='int64')

In [12]:
FG = DMG.loc[0, 1:] #fragility groups
n_PG = len(FG) #number of perfomance groups

DS=[]
DSG_DS = DMG.loc[3, 1:] #damage state groups
for i in range(len(DSG_DS)):
    DS.append(float(DSG_DS[i+1][0]))

RC_matrix = np.zeros((len(indx_repairable),n_PG))

In [ ]:
DSG_DS

In [15]:
DS

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
max(DS)

In [ ]:
len(RC_matrix)

In [13]:
#create DMG file for repairable realizations only
DMG_all = DMG.loc[4:, 1:]
for i in range(len(DMG_all)):
    if int(DMG.loc[i+4, 0]) in indx_irreparable:            
        DMG_all.iloc[i,:]=np.nan
DMG_repairable = DMG_all.dropna()
DMG_repairable = DMG_repairable.astype('float')
DMG_repairable = np.array(DMG_repairable)

In [14]:
DMG_repairable

array([[ 2.,  4.,  4., ...,  4., 39.,  8.],
       [ 2.,  4.,  4., ...,  4., 39.,  8.],
       [ 2.,  4.,  4., ...,  4., 39.,  8.],
       ...,
       [ 2.,  4.,  4., ...,  4., 39.,  8.],
       [ 2.,  4.,  4., ...,  4., 39.,  8.],
       [ 2.,  4.,  4., ...,  4., 39.,  8.]])

In [ ]:
len(DMG_repairable)

In [16]:
#create repair class matrix 将每个损伤类别赋予相应的维修等级
DS=pd.Series(DS)
for j in range(n_PG):
    indx_FG = FG_all[FG_all==FG[j+1]].index
    DS_num = DS[j]
    for i in range(len(indx_repairable)):
        if DMG_repairable[i,j] != 0:
            RC_matrix[i,j] = RC_all.iloc[indx_FG[0], int(DS_num)]

In [19]:
RC_matrix.shape

(2000, 3531)

In [22]:
RC_matrix4 = np.c_[np.arange(0,len(indx_repairable),1), RC_matrix] #np.c_[] 函数将两个数组按列连接
RC = pd.concat([DMG.loc[:3, :], pd.DataFrame(RC_matrix4)]) #repair class matrix consistant with DMG file 格式保持一致

RC.to_csv(os.path.join(output_path,r'RC_component.csv'), header=False, index=False)
print('Repair class matrix is generated')

Repair class matrix is generated


In [23]:
RC

,0,1,2,3,4,5,6,7,8,9,...,3522,3523,3524,3525,3526,3527,3528,3529,3530,3531
0,cmp,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,B.10.35.022,...,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.40.11.053a,D.50.11.013l,D.50.12.023c,D.50.12.023l
1,loc,1,1,10,10,11,11,12,12,13,...,6,60,60,60,7,8,9,28,28,28
2,dir,1,2,1,2,1,2,1,2,1,...,0,0,0,0,0,0,0,0,0,0
3,ds,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,0,0,0,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
#create a matrix that contains the maximum repair class per repair sequence per realization across the building 
RCmax_RS=[]
for i in range(len(indx_repairable)): 
    RC_realization = RC_matrix[i,:]
    RCmax_RS.append([])
    for j in range(len(story)):
        for k in range(len(RS)):
            indx_ST = list(locate(story_FG, lambda x: x == story[j]))
            indx_RS = list(locate(RS_FG, lambda x: x == RS[k]))
            indx_intersect = list(set.intersection(set(indx_ST),set(indx_RS)))
            RCmax_RS[i].append(np.max((RC_realization[indx_intersect]),initial=0)) #每个楼层每个维修顺序中的最大维修等级

print('Repair class calculation is completed')

Repair class calculation is completed


In [27]:
#需要保存的变量
# 定义pickle文件的路径
pickle_file_path = os.path.join(subdir_path, 'RC_results.pkl')

data_dict = {
    "RCmax_RS": RCmax_RS,
    "RC": RC,
    "indx_repairable": indx_repairable,
    "indx_irreparable": indx_irreparable,
    "indx_collapse": indx_collapse,
    "story": story
}

# 保存数据到pickle文件
with open(pickle_file_path, "wb") as file:
    pickle.dump(data_dict, file)